In [ ]:
%pip install numpy
%pip install sentencepiece protobuf
%pip install polars==1.38.1
%pip install python-dotenv==1.2.1
%pip install openai==2.16.0
%pip install readability-lxml==0.8.4.1 w3lib==2.4.0 httpx==0.28.1 beautifulsoup4==4.14.3 azure-ai-inference==1.0.0b9
%pip install sentence-transformers==5.2.2 tiktoken==0.12.0

%restart_python

In [ ]:
import os
import json
import asyncio
import numpy  as np
import scipy  as sp
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
from dotenv   import load_dotenv
from logging  import Logger, getLogger
from openai   import OpenAI

import polars as pl
import pandas as pd
from pyspark import StorageLevel
from pyspark.sql import types, Window
from pyspark.sql.functions import col
import pyspark.sql.functions as F

from pyspark.sql import SparkSession, DataFrame
from delta import configure_spark_with_delta_pip

from azure.ai.inference.models import SystemMessage, UserMessage
from sentence_transformers     import SentenceTransformer

from url_scraper import fetch_web

In [ ]:
builder = SparkSession.builder\
            .config("spark.sql.sources.commitProtocolClass", "org.apache.spark.sql.execution.datasources.SQLHadoopMapReduceCommitProtocol")\
            .config("spark.sql.parquet.output.committer.class", "org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter")\
            .config("spark.mapreduce.fileoutputcommitter.marksuccessfuljobs","false")\
            .config("spark.sql.adaptive.enabled", True)\
            .config("spark.sql.shuffle.partitions", "auto")\
            .config("spark.sql.adaptive.advisoryPartitionSizeInBytes", "100MB")\
            .config("spark.sql.adaptive.coalescePartitions.enabled", True)\
            .config("spark.sql.dynamicPartitionPruning.enabled", True)\
            .config("spark.sql.autoBroadcastJoinThreshold", "10MB")\
            .config("spark.sql.session.timeZone", "Asia/Tokyo")\
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")\
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
            .config("spark.databricks.delta.write.isolationLevel", "SnapshotIsolation")\
            .config("spark.databricks.delta.optimizeWrite.enabled", True)\
            .config("spark.databricks.delta.autoCompact.enabled", True)
            # Delta Lake 用の SQL コミットプロトコルを指定
            # Parquet 出力時のコミッタークラスを指定
            # Azure Blob Storage (ABFS) 用のコミッターファクトリを指定
            # '_SUCCESS'で始まるファイルを書き込まないように設定
            # AQE(Adaptive Query Execution)の有効化
            # パーティション数を自動で調整するように設定
            # シャッフル後の1パーティションあたりの最小サイズを指定
            # AQEのパーティション合成の有効化
            # 動的パーティションプルーニングの有効化
            # 小さいテーブルのブロードキャスト結合への自動変換をするための閾値調整
            # SparkSessionのタイムゾーンを日本標準時刻に設定
            # Delta Lake固有のSQL構文や解析機能を拡張モジュールとして有効化
            # SparkカタログをDeltaLakeカタログへ変更
            # Delta Lake書き込み時のアイソレーションレベルを「スナップショット分離」に設定
            # 書き込み時にデータシャッフルを行い、大きなファイルを生成する機能の有効化
            # 書き込み後に小さなファイルを自動で統合する機能の有効化


spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
# .env ファイルを読み込む
load_dotenv()

# 環境変数の取得
AI_FOUNDRY_ENDPOINT = os.environ.get("AI_FOUNDRY_ENDPOINT")
AI_FOUNDRY_API_KEY  = os.environ.get("AI_FOUNDRY_API_KEY")
AI_FOUNDRY_MODEL    = os.environ.get("AI_FOUNDRY_MODEL")
MAX_TOKENS          = os.environ.get("MAX_TOKENS")
TEMPERATURE         = os.environ.get("TEMPERATURE")
TOP_P               = os.environ.get("TOP_P")

# メモ：
# ウィジット経由でのパラメータの取得方法では、無条件にパラメータの型がStringに変換されてしまう
# そのため、受け取ったパラメータを適切に型変換する必要がある
MAX_TOKENS  = int(MAX_TOKENS)
TEMPERATURE = float(TEMPERATURE)
TOP_P       = float(TOP_P)


# 簡易デバッグ用
print(f'AI_FOUNDRY_ENDPOINT: {AI_FOUNDRY_ENDPOINT}')
print(f'AI_FOUNDRY_API_KEY:  {AI_FOUNDRY_API_KEY}')
print(f'AI_FOUNDRY_MODEL:    {AI_FOUNDRY_MODEL}')
print(f'MAX_TOKENS:          {MAX_TOKENS}')
print(f'TEMPERATURE:         {TEMPERATURE}')
print(f'TOP_P:               {TOP_P}')

In [ ]:
llmClient = OpenAI(base_url=AI_FOUNDRY_ENDPOINT, api_key=AI_FOUNDRY_API_KEY)
logger    = getLogger(__name__)
semaphore = asyncio.Semaphore(10)

# 例とするランディングページ
LP_URL   = "https://lp.br-lb.com/"

res_dict = await fetch_web(logger, semaphore, LP_URL)
print(res_dict)

### 前処理データ部

INPUT:
- LLMによる商品LPの分析結果
- ADIDの行動特徴テーブル

OUTPUT:
- [1, コホートキャプション数]なLPのスポット係数ベクトル

In [ ]:
messages  = []
messages.append(SystemMessage(content=(
				"あなたは商品LPの分析を行うマーケティングの専門家です。\n"
				"提供された商品情報を分析し、その商品が「最高に輝く具体的なシーン（適合）」と「全く役に立たない、あるいはミスマッチなシーン（不適合）」を洗い出してください。\n\n"
				
				"【重要な指示】\n"
                "出力するキーワードは、ベクトル検索のクエリとして使用されます。\n"
				"そのため、単一の一般名詞（例：「公園」「オフィス」）は **禁止** です。\n"
                "必ず **「場所＋状況」** または **「属性＋場所」** の複合キーワード（Micro-Context）を選定してください。\n\n"
    			"抽出する単語は、単なる一般名詞ではなく、「誰が・どこで・何をしているか」がありありと想像できるような、具体的かつ解像度の高いキーワードを選定してください。\n"
    			"特に「場所」に関しては、大分類（例：公園）ではなく、詳細な施設タイプ（例：ドッグラン、親水広場）や、利用目的が明確なスポット名を優先してください。\n\n"
                "LPに直接記載がなくても、商品の特性から論理的に推測されるシーンは積極的に広げて記述してください。\n\n"
                "・NG例: 「ジム」「キャンプ」「サラリーマン」\n"
                "・OK例: 「深夜の24時間ジム」「雨上がりのオートキャンプ場」「満員電車の通勤客」「コンセントのあるカフェ席」\n\n"

				"【出力形式（厳守）】\n"
				"回答は必ず以下のJSON形式のみを出力してください。\n"
				"各単語をキーとし、その関連度の強さ（重み）を 0.0〜1.0 の数値で値として設定してください。\n"
				"Markdown記法（```json 等）は含めず、生のJSONテキストのみを返してください。\n\n"
                "・Positive/Negative 共に、確信度の高い上位5〜10個程度を抽出してください。\n\n"
					
				"""
				{
					"positive": {"複合キーワードA": 0.89, "キーワードB": 0.70, ...},
					"negative": {"複合キーワードC": 0.91, ...},
				}
				"""
				"\n\n"
				
				"【分析の視点】\n"
				"--- positive（適合）：商品が必須となる、または魅力を最大化する文脈 ---\n"
				"1. 具体的な施設・スポット（Places）：\n"
				"   - 抽象的な「店」「屋外」はNG。\n"
				"   - 「24時間ジム」「オートキャンプ場」「コワーキングスペース」など、行動が特定できる施設名。\n"
				"2. 利用シーン・瞬間（Scenes）：\n"
				"   - 「通勤ラッシュ」「運動後のシャワー」「子供の寝かしつけ」など、具体的なタイムラインや状況。\n"
				"3. ターゲットの属性・状態（Traits）：\n"
				"   - 「健康志向」のような広い言葉より、「糖質制限中」「リモートワーク疲れ」など具体的な状態。\n\n"
                "4. 物理的適合 (Physical Fit):\n"
                "   - 商品のサイズ、電源、耐久性が、その場所の設備・環境と完璧に噛み合うか。\n"
                "   - マグネットでくっつく防水Bluetoothスピーカー  →  「ユニットバスの壁面」「雨の日のキャンプのタープ下」"
				"5. 心理的・行動的適合 (Contextual Fit):\n"
                "   - その場所にいる人の「特定の悩み」を解決するか。\n"
				"   - 周囲の音を消すデジタル耳栓  →  「いびきが気になるカプセルホテル」「瞑想に集中したいヨガスタジオの隅」"
				
				"--- negative（不適合）：商品の機能が死ぬ、または邪魔になる文脈 ---\n"
				"1. 阻害要因となる場所（Places）：\n"
				"   - 商品のスペック（大きさ、音、電源有無など）的に使えない場所（例：図書館、満員電車）。\n"
				"2. 無意味なシーン（Scenes）：\n"
				"   - その商品をあえて使う必要がない状況。\n\n"
                "3. 環境不適合 (Environmental Mismatch):\n"
                "   - 商品を使うには狭すぎる、暗すぎる、うるさすぎる、静かすぎる場所。\n"
                "4. マナー・ルール違反 (Social Mismatch):\n"
                "   - その場所でその商品を使うことが「白い目」で見られる、あるいは禁止されている。\n"
				
				"【重み（スコア）の基準】\n"
				"・1.0に近いほど：その傾向が非常に強い、確信度が高い\n"
				"・0.0に近いほど：関連性が薄い\n"
                "・1.0: 完全にその商品の独壇場である（または絶対に使用不可である）。\n"
                "・0.8: 非常に相性が良い（または強い懸念がある）。\n"
                "・0.5: 条件による（今回は出力対象外）。\n"
				"・positiveの場合：適合度の高さ\n"
				"・negativeの場合：不適合度の高さ（明確に避けるべき度合い）"
			)))
messages.append(UserMessage(content=json.dumps(res_dict, indent=4, ensure_ascii=False)))
response      = llmClient.chat.completions.create(
					messages=messages,
					tools=None,
					tool_choice=None,
					max_tokens=MAX_TOKENS,
					temperature=TEMPERATURE,
					top_p=TOP_P,
					model=AI_FOUNDRY_MODEL
				)

result_text   = response.choices[0].message.content
analysis_data = json.loads(result_text)
analysis_data

In [ ]:
# メモ：
# cohort.npzは以下のようなデータ構成になっている
# cohort.npz
#     |-- data              : 計算済みコホート係数行列
#     |-- indices           : データの位置指定子(列)
#     |-- indptr            : 行ごとのスライスインデックス
#     |-- shape             : コホート係数行列の形(ADIDのリスト × コホートキャプションのリスト)
#     |-- adid_list         : ADIDのリスト(列)
#     |-- business_codelist : コホートキャプションIDのリスト(行)
TARGET      = "/Volumes/stgadintedmpadintedi/featurestore/behaviorvector/cohort.npz"
npz         = np.load(TARGET, allow_pickle=True)
np_cohort   = sp.sparse.csr_matrix((npz["data"], npz["indices"], npz["indptr"]), shape=tuple(npz["shape"]))
np_adidlist = npz["adid_list"]
np_codelist = npz["business_codelist"]


# CODELISTに対応するキャプションの文脈行列を取得
CAPTION_CONTEXT_MATRIX = "cohort_caption_matrix.npz"
npz                    = np.load(CAPTION_CONTEXT_MATRIX, allow_pickle=True)
spots_matrix           = npz["data"]
relational_spots       = npz["business_placelist"]
dict_code2name         = npz["dict_code2name"].item()

# 簡易チェック
assert all(code in dict_code2name for code in np_codelist), "There are unregistered keys."
assert np.array_equal(np.array([dict_code2name[code] for code in np_codelist]), relational_spots), "Location list is inconsistent."

print(np_cohort.shape)
print(spots_matrix.shape)

In [ ]:
items_positive   = list(analysis_data['positive'].items())
items_negative   = list(analysis_data['negative'].items())
lp_keywords      = [key for key, val in items_positive] + [ key for key, val in items_negative]
lp_weights       = [val for key, val in items_positive] + [-val for key, val in items_negative]

model            = SentenceTransformer('cl-nagoya/ruri-v3-130m')
lp_vector        = np.array(lp_weights).reshape(1, -1)               # 1 × キーワード数M
lp_matrix        = model.encode(lp_keywords)                         # キーワード数M × 512
lp_coefficient   = lp_vector @ lp_matrix @ spots_matrix.T            # LPのスポット係数
lp_coefficient

### ADID毎のスコア・理由を算出

In [ ]:
MAX_RECORDS      = 10000
MAX_VALUE        = np.max( lp_coefficient)
MIN_VALUE        = np.min( lp_coefficient)
CORRECT_MEAN     = np.mean(lp_coefficient)
CORRECT_STDDEV   = np.std( lp_coefficient)
lp_coefficient   = 2 * (lp_coefficient - MIN_VALUE) / (MAX_VALUE - MIN_VALUE) - 1
# lp_coefficient   = (lp_coefficient - CORRECT_MEAN) / CORRECT_STDDEV

# ADID毎のスコアを算出
np_scored        = (np_cohort @ lp_coefficient.T).flatten()
indices          = np.argsort(np_scored)[::-1][:MAX_RECORDS]
sorted_adidlist  = np_adidlist[indices]
sorted_spots     = relational_spots
sorted_targets   = np_cohort[indices, :]
sorted_scored    = np_scored[indices]

# 閾値以上のスポットを理由とする
REASON_THRESHOLD = 0.5
# REASON_THRESHOLD = 1.28
np_threshold     = lp_coefficient > REASON_THRESHOLD
np_reasons       = sorted_targets.multiply(np_threshold)
np_reasons.eliminate_zeros()
pldf_reasons     = pl.DataFrame({
    						'ADID'           : sorted_adidlist[np_reasons.row], 
                            'cohort_caption' : sorted_spots[np_reasons.col], 
                            'score'          : sorted_scored[np_reasons.row], 
                            'value'          : np_reasons.data
                        })\
						.filter(pl.col('value') > 0)\
                        .group_by(pl.col('ADID'), maintain_order=True)\
                        .agg(
                            pl.col('score').first(),
                            pl.col('cohort_caption').str.join(', ').alias('reasons')
                        )\
                        .select(['ADID', 'score', 'reasons'])


print(pldf_reasons)

In [ ]:
extracted_spots = sorted_spots[np_threshold.flatten()]
messages        = []
messages.append(SystemMessage(content=(
				"あなたは、複数の場所・施設名から、それらを行動範囲とする人物の「ライフスタイル」や「価値観」を逆引きで特定するプロファイリングの専門家です。\n"
				"**入力された全ての場所・施設を利用する可能性が高い人物像**を分析し、その共通項（Intersection）から浮かび上がる具体的なペルソナを3人描き出してください。\n\n"

				"【出力形式（厳守）】\n"
				"回答は必ず以下のJSON形式のみを出力してください。\n"
				"キーには「persona_1」「persona_2」...を使用し、値に分析結果のテキストを入れてください。\n"
				"Markdown記法（```json 等）は含めず、生のJSONテキストのみを返してください。\n\n"
					
				"""
				{
					"persona_1": "分析された具体的なペルソナ像A（属性＋状況＋行動）",
        			"persona_2": "分析された具体的なペルソナ像B（属性＋状況＋行動）",
        			"persona_2": "分析された具体的なペルソナ像B（属性＋状況＋行動）"
				}
				"""
				"\n\n"
				
				"【分析プロセス（重要）】\n"
				"1. 点をつなぐ: 入力された施設A、施設B、施設C...の全てに親和性があるのはどのような人物か？\n"
				"2. 背景を読む: なぜその人はそれらの場所を選ぶのか？（共通するニーズ、美意識、所得層、時間の使い方）\n"
				"3. 解像度を上げる: 下記のルールに従い、具体的な情景として出力する。\n\n"

				"【抽出・生成のルール】\n"
				"1. 語彙の解像度（Micro-Context）\n"
				"   - 単なる属性名ではなく、「属性＋具体的な状況＋心理/行動」のセットで記述してください。\n"
				"   - 抽象的な表現（例：富裕層、サラリーマン）は **禁止** です。\n"
				"   - NG: 「週末にジムとカフェに行く会社員」\n"
				"   - OK: 「週末の午前中にパーソナルジムで汗を流した後、ラウンジでプロテインを飲みながら投資信託のチェックをするアッパーマス層」\n\n"

				"2. 多角的な視点の統合\n"
				"   - 複数の施設に共通する「空気感」や「質」へのこだわりを盛り込む。\n"
				"   - 【時間軸】: 彼らはいつ現れるか（早朝のルーティン、深夜の逃避など）\n"
				"   - 【五感・嗜好】: 彼らが好む環境（静寂、活気、オーガニック、ラグジュアリーなど）\n\n"
				
				"3. 文体\n"
				"   - 「〜です」「〜ます」は不要。体言止めや現在進行形で、その人物の生活を切り取るように記述する。\n"
				"   - 1文あたり120文字程度。"
			)))
messages.append(UserMessage(content=json.dumps(res_dict, indent=4, ensure_ascii=False)))
response     = llmClient.chat.completions.create(
					messages=messages,
					tools=None,
					tool_choice=None,
					max_tokens=MAX_TOKENS,
					temperature=TEMPERATURE,
					top_p=TOP_P,
					model=AI_FOUNDRY_MODEL
				)

result_text  = response.choices[0].message.content
persona_data = json.loads(result_text)
persona_data

In [ ]:
extracted_spots = sorted_spots[np_threshold.flatten()]
messages        = []
messages.append(SystemMessage(content=(
				"あなたは、消費者心理を深く洞察し、売れる文脈を設計する「マーケティング・ストラテジスト」です。\n"
    			"提供された「商品LP情報」と「ターゲットペルソナ」を分析し、そのターゲットの心を動かす**具体的な利用想起シナリオ（訴求アングル）**を2パターン開発してください。\n\n"

				"【出力形式（厳守）】\n"
				"回答は必ず以下のJSON形式のみを出力してください。\n"
				"Markdown記法（```json 等）は含めず、生のJSONテキストのみを返してください。\n\n"
					
				"""
				{
					"scenario_1": {
						"title":          "シナリオのタイトル（コンセプト名）",
						"target_insight": "ペルソナが抱えている隠れた本音・悩み（インサイト）",
						"product_match":  "LP内のどの要素が解決策になるか（具体的な機能・特徴）",
						"usage_scene":    "商品を利用している、または商品を欲しくなる具体的な情景描写（Micro-Context）",
						"catchphrase":    "そのペルソナに刺さるキャッチコピー"
					},
        			"scenario_2": { ... }
				}
				"""
				"\n\n"
				
				"【分析プロセス（重要）】\n"
				"1. ペルソナの生活を憑依させる: その人は普段どんな場所にいて、どんな悩みを抱え、何を求めているか？\n"
    			"2. 接点を見つける:          商品のどの機能（スペック）が、そのペルソナのどの課題（ペイン）を解決するか？\n"
    			"3. シチュエーションを描く:    その商品が最も魅力的に映る「具体的な瞬間」を切り取る。\n\n"

				"【品質基準】\n"
                "- 表面的なマッチング（例：太っているからジムへ）は禁止。\n"
				"- 「なぜそのペルソナなのか」が論理的に紐付いていること。\n"
				"- 情景描写は五感に訴えるレベル具体的に（例：「残業終わりの重い足取りで...」）。\n"
				"- 商品情報は必ず入力されたLPデータに基づいていること（捏造しない）。"
			)))
user_input_text = f"""
【分析対象データ】

1. 商品LP情報 (Product):
{json.dumps(res_dict,     indent=4, ensure_ascii=False)}

2. ターゲットペルソナ (Target Persona):
{json.dumps(persona_data, indent=4, ensure_ascii=False)}

この商品とペルソナの「運命的な出会い」を演出するシナリオを作成してください。
"""
messages.append(UserMessage(content=user_input_text))
response     = llmClient.chat.completions.create(
					messages=messages,
					tools=None,
					tool_choice=None,
					max_tokens=MAX_TOKENS,
					temperature=TEMPERATURE,
					top_p=TOP_P,
					model=AI_FOUNDRY_MODEL
				)

result_text  = response.choices[0].message.content
appeal_scenario_data = json.loads(result_text)
appeal_scenario_data